In [103]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from pathlib import Path 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
plt.style.use('fivethirtyeight')
import wikipediaapi
from pathlib import Path
import time
from time import sleep

import csv
import requests
from bs4 import BeautifulSoup
import urllib.request, urllib.parse, urllib.error
import ssl

In [5]:
path = Path('./the_movie_dataset')
files = [p for p in path.glob('*.csv')]
files
credits = pd.read_csv([f for f in files if 'credits' in str(f)][0], dtype={'id':'str'})
credits.rename({'id':'tmdbId'},axis=1, inplace=True)
print(credits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   tmdbId  45476 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB
None


In [6]:
# credits['cast_main_5'] = ( credits['cast'].apply(ast.literal_eval)
#                             .apply(lambda x: [ x['name'] for x in x[:5]] ) )
# credits['cast_main_5_gender'] = ( credits['cast'].apply(ast.literal_eval)
#                             .apply(lambda x: [ x['gender'] for x in x[:5]] ) )
# credits[['actor_1_name',
#          'actor_2_name',
#          'actor_3_name',
#          'actor_4_name',
#          'actor_5_name']] = ( credits['cast_main_5']
#                              .apply(lambda x: pd.Series(x)) )


# credits[['actor_1_gender',
#          'actor_2_gender',
#          'actor_3_gender',
#          'actor_4_gender',
#          'actor_5_gender']] = ( credits['cast_main_5_gender']
#                              .apply(lambda x: pd.Series(x)) )

credits['cast_main_5'] = ( credits['cast'].apply(ast.literal_eval)
                            .apply(lambda x: [ x['name'] for x in x[:3]] ) )
credits['cast_main_5_gender'] = ( credits['cast'].apply(ast.literal_eval)
                            .apply(lambda x: [ x['gender'] for x in x[:3]] ) )

In [46]:
credits

,cast,crew,tmdbId,cast_main_5,cast_main_5_gender
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[Tom Hanks, Tim Allen, Don Rickles]","[2, 2, 2]"
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[2, 2, 1]"
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"[Walter Matthau, Jack Lemmon, Ann-Margret]","[2, 2, 1]"
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,"[Whitney Houston, Angela Bassett, Loretta Devine]","[1, 1, 1]"
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"[Steve Martin, Diane Keaton, Martin Short]","[2, 1, 2]"
...,...,...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,"[Leila Hatami, Kourosh Tahami, Elham Korda]","[1, 0, 0]"
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,"[Angel Aquino, Perry Dizon, Hazel Orencio]","[1, 0, 0]"
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,"[Erika Eleniak, Adam Baldwin, Julie du Page]","[1, 2, 1]"
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...","[2, 1, 2]"


In [7]:
credits[['actor_1_name',
         'actor_2_name',
         'actor_3_name']] = ( credits['cast_main_5']
                             .apply(lambda x: pd.Series(x)) )


credits[['actor_1_gender',
         'actor_2_gender',
         'actor_3_gender']] = ( credits['cast_main_5_gender']
                             .apply(lambda x: pd.Series(x)) )

<ipython-input-7-9db9bf101b1f>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(lambda x: pd.Series(x)) )
<ipython-input-7-9db9bf101b1f>:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(lambda x: pd.Series(x)) )


In [74]:

unique_actors = set([item for sublist in credits['cast_main_5'].to_list() for item in sublist])
unique_actors = dict.fromkeys(unique_actors, (0,0)) 
list(unique_actors)[:5]

['Aki Avni',
 'Coen van Vrijberghe de Coningh',
 'Jim Driscoll',
 'Raivo E. Tamm',
 'Rasmus Bjerg']

In [75]:
len(unique_actors)

47481

In [6]:
unique_actors

{'Jarlath Conroy': (0, 0),
 'Colin Moy': (0, 0),
 'Chantal Barré': (0, 0),
 'Kenny Wormald': (0, 0),
 'Joanna David': (0, 0),
 'Terence De Marney': (0, 0),
 'Marilyn Bautista': (0, 0),
 'Dorota Stalińska': (0, 0),
 'Jocelyn Towne': (0, 0),
 'Si Newhouse IV': (0, 0),
 'Kichiemon Nakamura': (0, 0),
 'Abdul Salis': (0, 0),
 'Chloe Levine': (0, 0),
 'Nada Abou Farhat': (0, 0),
 'Ray Valenti': (0, 0),
 'Earl Hindman': (0, 0),
 'Lynn Monteil': (0, 0),
 'Tasha Smith': (0, 0),
 'Natasha Morley': (0, 0),
 'Gabriella Pession': (0, 0),
 'Park Hyuk-kwon': (0, 0),
 'William Alva': (0, 0),
 'Josh Groban': (0, 0),
 'John Gordon Sinclair': (0, 0),
 'Madonna Sebastian': (0, 0),
 'Ansiba Hassan': (0, 0),
 'Kitty Aldridge': (0, 0),
 'John Benjamin Martin': (0, 0),
 'Misa Jokisalo': (0, 0),
 'Riccardo Billi': (0, 0),
 'Scott Beehner': (0, 0),
 'Krsna Angulo': (0, 0),
 'Jose Yenque': (0, 0),
 'Matty Liu': (0, 0),
 'Phil Vassar': (0, 0),
 'Kiana Cason': (0, 0),
 'Richard Norton': (0, 0),
 'Zvezda Angelovska

In [76]:
df = pd.DataFrame(unique_actors).T.rename({0:'won',1:'nominat'},axis=1).reset_index()

In [77]:
df.to_csv('actors.csv')

In [82]:
df

,index,won,nominat
0,Aki Avni,0,0
1,Coen van Vrijberghe de Coningh,0,0
2,Jim Driscoll,0,0
3,Raivo E. Tamm,0,0
4,Rasmus Bjerg,0,0
...,...,...,...
47476,Eddie Mills,0,0
47477,Sirisin Siripornsmathikul,0,0
47478,Marleen Lohse,0,0
47479,Silvio Hernández,0,0


In [99]:
paralle = np.linspace(0,df.shape[0],4)
paralle

array([    0., 15827., 31654., 47481.])

In [101]:
df_sub = df.iloc[:int(paralle[1])]
df_sub

,index,won,nominat
0,Aki Avni,0,0
1,Coen van Vrijberghe de Coningh,0,0
2,Jim Driscoll,0,0
3,Raivo E. Tamm,0,0
4,Rasmus Bjerg,0,0
...,...,...,...
15822,Paul Butcher,0,0
15823,Roger Gobeth,0,0
15824,Marcus Hearn,0,0
15825,Antonella Ríos,0,0


In [104]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

stats = time.time()
exceptions = 'List_of_awards_and_nominations_received_by_'
with open('actor_awards_1.csv','w') as csvfile:
    fieldnames = ['name','won','nominat']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()


    for i,n in enumerate(df_sub['index']):
        
        # print progress
        _, r = divmod(i,1000)
        if r == 0:
            print(i,df_sub.shape[0])
        
        name = n.replace(' ','_')
        sleep(np.round(np.random.random_sample(),3))
        try : # 수상 리스트가 따로 있을 때
            url = f'https://en.wikipedia.org/wiki/{exceptions}{name}'
            html = urllib.request.urlopen(url, context=ctx).read()
        except :

            try: # 따로 없을 때, 이름으로 찾기
                url = f'https://en.wikipedia.org/wiki/{name}'
                html = urllib.request.urlopen(url, context=ctx).read()
            except: # 이름이 없을 때는 방법 없음
                cnt_won = np.nan
                cnt_nomi = np.nan
                continue
                
        # html parser 
        soup = BeautifulSoup(html, 'html.parser')
        t =soup.find(id='bodyContent').text
        cnt_won = t.lower().count('won')
        cnt_nomi = t.lower().count('nominat')
        
        writer.writerow({'name':n
                          ,'won':cnt_won
                          ,'nominat':cnt_nomi
                         })
print(time.time()-stats)

0 15827
1000 15827
2000 15827
3000 15827
4000 15827
5000 15827
6000 15827
7000 15827
8000 15827
9000 15827
10000 15827
11000 15827
12000 15827
13000 15827
14000 15827
15000 15827
39653.230185985565


In [69]:
for i, n in df['index'][:20]:
    name = n.replace(' ','_')
    url = f'https://en.wikipedia.org/wiki/{exceptions}{name}'
html = urllib.request.urlopen(url, context=ctx).read()

In [70]:
html

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Lewis Fitz-Gerald - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"YDO8aVv3412jCKBGYA3oEwAAABg","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Lewis_Fitz-Gerald","wgTitle":"Lewis Fitz-Gerald","wgCurRevisionId":997797679,"wgRevisionId":997797679,"wgArticleId":1620621,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Articles needing additional references from June 2018","All articles needing a

In [71]:
soup = BeautifulSoup(html, 'html.parser')

In [73]:
t = soup.find(id='bodyContent').text
t

'\nFrom Wikipedia, the free encyclopedia\n\n\n\nJump to navigation\nJump to search\nAustralian actor, screenwriter and television director\nThis article needs additional citations for verification. Please help improve this article by adding citations to reliable sources. Unsourced material may be challenged and removed.Find sources:\xa0"Lewis Fitz-Gerald"\xa0–\xa0news\xa0· newspapers\xa0· books\xa0· scholar\xa0· JSTOR (June 2018) (Learn how and when to remove this template message)\n\n\nLewis Fitz-GeraldBorn (1958-11-15) 15 November 1958 (age\xa062)Adelaide, South AustraliaAlma\xa0materUniversity of New England (NSW)National Institute of Dramatic Art (NIDA)OccupationActorwriterdirectoracademicYears\xa0active1979–presentWebsiteLewis Fitz-Gerald\nLewis Fitz-Gerald (born 15 November 1958)[1] is an Australian actor, screenwriter and television director. His academic qualifications include PhD (UNE), MA (Comms) UNE, BDA (Acting) NIDA. Fitz-Gerald lectures in Screen and Media Studies at Aust

In [47]:
s = soup.select("div#bodyContent")
for ss in s.find_all('div'):
    print('n')

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [42]:
s = soup.select("div#bodyContent")
for ss in s.find("div"):
    print(ss.text)
#     for sss in ss.find("div"):
#         print('sep==============')
#         print(sss.text)

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [19]:
where = [10000,20000,30000,40000,50000,60000,70000,80000,90000]
n=0
start = time.time()
for i,name in enumerate(df['index'][:1000]):
    if i > 1:
        # print progress
        _, r = divmod(i,1000)
        if r == 0:
            print(i)
            n +=1
        
    
    try :
    
        page = wiki.page(name)
    
        if len(page.categories) < 5:
            name = name + ' (actor)'
            page = wiki.page(name) # re-searching
            
            if len(page.categories) < 5: # if not find retun fail
                df.loc[df['index']==name,'won'] = 'fail'
                df.loc[df['index']==name,'nominat'] = 'fail'
            else :
                t = page.text # if (actor) find some 
                df.loc[df['index']==name,'won'] = t.count('won')
                df.loc[df['index']==name,'nominat'] = t.count('nominat')

        else : 
            t = page.text
            df.loc[df['index']==name,'won'] = t.count('won')
            df.loc[df['index']==name,'nominat'] = t.count('nominat')

    except : # if error 
        
        df.loc[df['index']==name,'won'] = 'error'
        df.loc[df['index']==name,'nominat'] = 'error'
        
        continue # next roop
print(time.time()-start)

4.6093409061431885


In [20]:
df[(df['won']!='error') & (df['won']!=0)]

,index,won,nominat


In [ ]:
unique_actors

In [ ]:
tmp_save = pd.Series(unique_actors)

In [1]:
tmp_save

NameError: name 'tmp_save' is not defined